# Modelos probabilísticos (Ejercicio)

## Aplicación de Naive Bayes multinomial a la detección de SMS *spam*

En este ejercicio se pide reproducir lo realizado en el caso práctico que se ha descrito en los vídeos (análisis de sentimiento en críticas de cine), pero ahora para detectar cuándo un mensaje corto (SMS) es *spam*.

### El conjunto de datos

El conjunto de datos consiste una serie de mensajes SMS (5574 en total), que están clasificados como mensajes basura (*spam*) o mensajes normales (*ham*). Los datos se pueden obtener en el [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

En concreto, descargar el fichero [smsspamcollection.zip](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip), y descomprimirlo para obtener un fichero de texto SMSSpamCollection. En este fichero de texto hay una línea por cada sms, con el formato: *clase* *tabulador* *sms*. Por ejemplo, la primera línea es:

`ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...`

El fichero debe ser leído convenientemente para poder aplicar la vectorización. Se puede hacer la lectura usando las funciones python de lectura de ficheros, pero se recomienda usar la instrucción `read_table` de la biblioteca `pandas`:

In [128]:
import pandas as pd

*Pandas* es una biblioteca de python muy utilizada para manipular y analizar datos. Si el fichero se lee con la orden `read_table` (se pide averiguar la manera concreta de hacerlo), entonces se obtendrá una tabla (o *Data Frame*), en el que las etiquetas serán una columna y los correspondientes sms otra. Esto permite obtener de manera sencilla la lista de etiquetas o clases, y por otro lado la lista de mensajes, en el mismo orden.  

### Aprendiendo a clasificar SMSs

Se pide reproducir con estos datos lo realizado en el *notebook* en el que se aplica Naive Bayes Multinomial al análisis de sentimientos de críticas de cine, pero ahora para clasificar un SMS como *spam* o como normal. Esto incluye:

* Separación de los textos en entrenamiento y prueba 
* Vectorización de los textos 
* Aprendizaje con `MultinomialNB`
* Mostrar algunas clasificaciones sobre sms concretos.
* Rendimiento sobre entrenamiento y prueba.
* Ajuste manual del parámetro de suavizado
* Vectorización con `min_df` y `stop_words` 

**Nota**: este conjunto de datos no es balanceado (la mayoría son *ham*). Por tanto, usar `score` no es muy ilustrativo del rendimiento, ya que un clasificador "tonto" que siempre predijera *ham* tendría un rendimiento alto. Por ello, en este caso también se hace necesario usar el método `confusion_matrix` del módulo `metrics`. Se pide también explicar la salida que proporciona dicha métrica.

Se pide **comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría. En particular, se pide mostrar parte de los atributos `class_count_`, `class_log_prior_`, `feature_count_` y `feature_log_prob_`, explicando claramente qué son cada uno de ellos. Explicar también cómo realiza las predicciones el modelo aprendido, tal y como se ha explicado en la teoría.  



# Inicio del ejercicio

## Separación de los textos en entrenamiento y prueba

Primero leemos el archivo de datos con el comando read_table de pandas, como se sugiere en el enunciado. Para ello ejecutamos el métidi pasandole como parámetros el nombre del archivo de datos y, en este caso el parámetro header=None para que no de nombre a las columnas.

In [129]:
df=pd.read_table("SMSSpamCollection", header=None)

Ahora doy los nombres X e Y a las columnas del dataframe para que sea más cómodo trabajar con ellas y creo las variables x e y respectivamente.

In [130]:
df.columns = ["Y","X"]

In [131]:
x=df.get('X')
y=df.get('Y')

A continuación importo el método trains_test_split de la librería sklearn para separar los textos en entrenamiento y prueba.

In [132]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.33,
                   random_state=4861,stratify=y)

In [133]:
print(len(x_train), len(y_train), len(x_test), len(y_test))

3733 3733 1839 1839


Ahora ya tengo los conjuntos de entrenamiento y prueba para trabajar.

## Vectorización de los textos

Ahora vectorizamos las variables x_train y x_test con countVectorizer de sklearn

In [134]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(x_train)

In [135]:
xf_train = vect.transform(x_train)
xf_test = vect.transform(x_test)

Ahora tenemos las nuevas variables xf_train y xf_test qu están en forma vectorizada.

In [136]:
print("yf_train:\n{}".format(repr(xf_train)))

yf_train:
<3733x7053 sparse matrix of type '<class 'numpy.int64'>'
	with 49319 stored elements in Compressed Sparse Row format>


## Aprendizaje con MultinomialNB

El algoritmo naives bayes multinomial, que es el que usamos aquí, utiliza el número de veces que las palabras aparecen en los documentos.

Ahora entrenamos un modelo con los datos de entrenamiento:

In [137]:
from sklearn.naive_bayes import MultinomialNB
multinb=MultinomialNB().fit(xf_train,y_train)

A continuación se muestran los atributos del modelo que acabamos de entrenar:

In [138]:
print("class_count_:\n{}".format(multinb.class_count_))
print("class_log_prior_:\n{}".format(multinb.class_log_prior_))
print("feature_count_:\n{}".format(multinb.feature_count_))
print("feature_log_prob_:\n{}".format(multinb.feature_log_prob_))

class_count_:
[3233.  500.]
class_log_prior_:
[-0.1438017  -2.01035938]
feature_count_:
[[ 0.  0.  1. ...  1.  1.  0.]
 [ 5. 15.  0. ...  0.  0.  1.]]
feature_log_prob_:
[[-10.79616159 -10.79616159 -10.10301441 ... -10.10301441 -10.10301441
  -10.79616159]
 [ -8.04841548  -7.06758622  -9.84017495 ...  -9.84017495  -9.84017495
   -9.14702777]]


* `class_count_`: Indica el número de casos que tenemos para cada clasificación. En este caso el conjunto no está balanceado, ya que tenemos muchos mas ejemplos de un caso que del otro.
* `class_log_prior_`: Indica la probabilidad de pertenecer a cada clase, suavizada usando el logaritmo.
* `feature_count_`: Indica, para cada clase, la cantidad de apariciones de cada palabra.
* `feature_log_prob_`: Indica, para cada clase, la probabilidad de que cada palabra aparezca o no en un texto.

Este tipo de clasificador predice con la clase a la que se que tiene una mayor probabilidad de pertenecer, utilizando las probabilidades de las palabras que aparecen en el caso que trata de predecir. Dichas probabilidades se peuden ver en `feature_log_prob_`, como se ha explicado arriba.

## Algunos SMS concretos

In [139]:
print("SMS 24 del conjunto test: \n\n{}\n".format(x_test.reset_index(drop=True).loc[24]))
print("Clasificación verdadera: {}.\n\n".format(y_test.reset_index(drop=True)[24]))

print("SMS 247 del conjunto test: \n\n{}\n".format(x_test.reset_index(drop=True)[247]))
print("Clasificación verdadera: {}.\n\n".format(y_test.reset_index(drop=True)[247]))

SMS 24 del conjunto test: 

Nope. Since ayo travelled, he has forgotten his guy

Clasificación verdadera: ham.


SMS 247 del conjunto test: 

Watching tv now. I got new job :)

Clasificación verdadera: ham.




In [140]:
print("Predicción del clasificador para SMS 24: {}\n".format(multinb.predict(vect.transform([x_test.reset_index(drop=True)[24]]))[0]))

print("Predicción del clasificador para SMS 247: {}".format(multinb.predict(vect.transform([x_test.reset_index(drop=True)[247]]))[0]))

Predicción del clasificador para SMS 24: ham

Predicción del clasificador para SMS 247: ham


Podemos ver como ambos ejemplos han sido clasificados de manera correcta.

## Rendimiento sobre entrenamiento y prueba

In [141]:
print("Rendimiento de multinb sobre el conjunto de entrenamiento: {:.2f}".format(multinb.score(xf_train,y_train)))
print("Rendimiento de multinb sobre el conjunto de test: {:.2f}".format(multinb.score(xf_test,y_test)))

Rendimiento de multinb sobre el conjunto de entrenamiento: 0.99
Rendimiento de multinb sobre el conjunto de test: 0.99


#### Como el conjunto de datos no está balanceado y un clasificardor tonto que siempre predijese 'ham' conseguiría un score alto, utilizaremos además el método confusion_matrix del módulo metrics de sklearn

In [142]:
from sklearn.metrics import confusion_matrix
y_true = y_test.values
y_pred = multinb.predict(xf_test)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = ['ham','spam']).ravel()
(tn, fp, fn, tp)

(1584, 8, 17, 230)

De la matriz de confusión obtenemos los valores de __verdaderos negativos__, __falsos positivos__, __falsos negativos__ y __verdaderos positivos__.

## Ajuste manual del parámetro de suavizado

In [143]:
multinb_alpha1 = MultinomialNB(alpha = 10).fit(xf_train,y_train)
multinb_alpha2 = MultinomialNB(alpha = 3).fit(xf_train,y_train)
multinb_alpha3 = MultinomialNB(alpha = 1.5).fit(xf_train,y_train)
multinb_alpha4 = MultinomialNB(alpha = 0.9).fit(xf_train,y_train)
multinb_alpha5 = MultinomialNB(alpha = 0.4).fit(xf_train,y_train)
multinb_alpha6 = MultinomialNB(alpha = 0.1).fit(xf_train,y_train)
multinb_alpha7 = MultinomialNB(alpha = 0.01).fit(xf_train,y_train)

ya1_pred = multinb_alpha1.predict(xf_test)
ya2_pred = multinb_alpha2.predict(xf_test)
ya3_pred = multinb_alpha3.predict(xf_test)
ya4_pred = multinb_alpha4.predict(xf_test)
ya5_pred = multinb_alpha5.predict(xf_test)
ya6_pred = multinb_alpha6.predict(xf_test)
ya7_pred = multinb_alpha6.predict(xf_test)

In [144]:
l_pred = [ya1_pred,ya2_pred,ya3_pred,ya4_pred,ya5_pred,ya6_pred,ya7_pred]
con_list = [confusion_matrix(y_true, x, labels = ['ham','spam']).ravel() for x in l_pred]
for x in con_list:
    res = [x[1]+x[2]]
    print(x,res)
#tn, fp, fn, tp = confusion_matrix(y_true, ya_pred, labels = ['ham','spam']).ravel()
#(tn, fp, fn, tp)

[1591    1   81  166] [82]
[1589    3   30  217] [33]
[1585    7   18  229] [25]
[1583    9   16  231] [25]
[1580   12   11  236] [23]
[1581   11   11  236] [22]
[1581   11   11  236] [22]


Podemos obsrevar que el número de casos mal clasificados se reduce en conjunto al disminuir el parámetro alfa, y que en varias ocasiones se produce un desplazamiento de los casos mal clasificados entre los falsos positivos y los falsos negativos.

## Vectorización con min_df y stop_words

In [145]:
vect2 = CountVectorizer(min_df=50, stop_words="english").fit(x_train)
xf2_train = vect2.transform(x_train)

Hemos creado un segundo vectorizador utilizando en este caso las opciones de **min_df**, que establece un número mínimo de paraiciones en el texto para considerar una palabra como relevante, y **stop_words**, que recibe el nombre de una lista  de palabras que eliminará por ser irrelevantes.

In [146]:
feature_names = vect.get_feature_names()
print("Número de términos en el vocabulario original: {}".format(len(feature_names)))
feature_names2 = vect2.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df: {}".format(len(feature_names2)))

Número de términos en el vocabulario original: 7053
Número de términos en el vocabulario con stop words y min_df: 75


Ahora creamos un nuevo clasificador utilizando el nuevo valor vectorizado:

In [147]:
multinb2=MultinomialNB(alpha=1).fit(xf2_train,y_train)

xf2_test = vect2.transform(x_test)
y2_pred = multinb2.predict(xf2_test)
y2_true = y_test.values

tn, fp, fn, tp = confusion_matrix(y2_true, y2_pred, labels = ['ham','spam']).ravel()
(tn, fp, fn, tp)

(1572, 20, 59, 188)

Se ha comprobado que el valor máximo que se puede utilizar en **min_df** en este caso es **235**. Para dicho valor el clasificador no es capaz de reconocer los casos negativos demanera correcta y clasifica todo como positivo.

In [148]:
vect235 = CountVectorizer(min_df=235, stop_words="english").fit(x_train)
xf235_train = vect235.transform(x_train)
multinb235=MultinomialNB(alpha=1).fit(xf235_train,y_train)

xf235_test = vect235.transform(x_test)
y235_pred = multinb235.predict(xf235_test)
tn, fp, fn, tp = confusion_matrix(y2_true, y235_pred, labels = ['ham','spam']).ravel()
(tn, fp, fn, tp)

(1592, 0, 247, 0)

Y, por otro lado, el número para el que se ha observado el menor valor, de falsos negativos y falsos positivos es 1, de manera que no se toman en cuenta las palabras que sólo aparecen 1 vez.

In [149]:
vect100 = CountVectorizer(min_df=1, stop_words="english").fit(x_train)
xf100_train = vect100.transform(x_train)
multinb100=MultinomialNB(alpha=1).fit(xf100_train,y_train)

xf100_test = vect100.transform(x_test)
y100_pred = multinb100.predict(xf100_test)
tn, fp, fn, tp = confusion_matrix(y2_true, y100_pred, labels = ['ham','spam']).ravel()
(tn, fp, fn, tp)

(1581, 11, 15, 232)

Comparamos la diferencia en los vocabularios de min_def = 235 y min_df = 1:

In [150]:
feature_names235 = vect235.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df = 235: {}".format(len(feature_names235)))
feature_names100 = vect100.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df = 1: {}".format(len(feature_names100)))

Número de términos en el vocabulario con stop words y min_df = 235: 1
Número de términos en el vocabulario con stop words y min_df = 1: 6794


Ahora comprobamos que ocurre si no utilizamos min_df:

In [151]:
vectst = CountVectorizer(stop_words="english").fit(x_train)
xfst_train = vectst.transform(x_train)
multinbst=MultinomialNB(alpha=1).fit(xfst_train,y_train)

xfst_test = vectst.transform(x_test)
yst_pred = multinbst.predict(xfst_test)
tn, fp, fn, tp = confusion_matrix(y2_true, yst_pred, labels = ['ham','spam']).ravel()
(tn, fp, fn, tp)

(1581, 11, 15, 232)

In [152]:
feature_names100 = vect100.get_feature_names()
print("Número de términos en el vocabulario con stop words y min_df = 1: {}".format(len(feature_names100)))
feature_names_st = vectst.get_feature_names()
print("Número de términos en el vocabulario con stop words y sin min_df: {}".format(len(feature_names_st)))

Número de términos en el vocabulario con stop words y min_df = 1: 6794
Número de términos en el vocabulario con stop words y sin min_df: 6794


Se puede observar que, en nuestro caso, un mayor valor de min_df aumenta la cantidad de casos clasificados de manera errónea, y que para el valor que se han observado mejores resultados, que es 1, se obtienen los mismos resultados y el mismo tamaño de vocabulario que si optamos por no utilizar el parámetro min_df y sólo usar **stop_words**.